# GNN Inference on Google Vertex AI using TigerGraph

In this notebook, we will train a GNN model and deploy it to Google Vertex AI as an inference endpoint.

## Setup

We are going to create a working directory.
**Note:** the `mkdir` command below will fail if the directory already exists. You can safely ignore the error message.

In [ ]:
import os

source_directory = "gat_cora"

os.mkdir("./{}".format(source_directory))

## Define The Model

We are going to define a Graph Attention Network (GAT) model, and write it to a file called `model.py`.

In [ ]:
%%writefile $source_directory/model.py

import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GAT(torch.nn.Module):
    def __init__(
        self, num_features, num_layers, out_dim, dropout, hidden_dim, num_heads
    ):
        super().__init__()
        self.dropout = dropout
        self.layers = torch.nn.ModuleList()
        for i in range(num_layers):
            in_units = num_features if i == 0 else hidden_dim * num_heads
            out_units = out_dim if i == (num_layers - 1) else hidden_dim
            heads = 1 if i == (num_layers - 1) else num_heads
            self.layers.append(
                GATConv(in_units, out_units, heads=heads, dropout=dropout)
            )

    def reset_parameters(self):
        for layer in self.layers:
            layer.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x.float(), data.edge_index
        for layer in self.layers[:-1]:
            x = layer(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.layers[-1](x, edge_index)
        return x

## Model Parameters

Here, we define a dictionary of the parameters of the model, data loaders, and connection to the database.

In [ ]:
parameters = {
    "model_name": "GAT",
    "model_config": {
        "num_features": 1433, # Number of features on Cora vertices 
        "out_dim": 7,         # Number of classes in Cora
        "num_heads": 8,       # Number of attention heads in GAT model
        "hidden_dim": 8,      # Number of hidden units in GAT model
        "num_layers": 2,      # Number of GAT layers in GAT model
        "dropout": 0.6        # Dropout probability in GAT model
    },
    "infer_loader_config": {
        "v_in_feats": ["x"],     # List of vertex features to be loaded
        "v_out_labels": ["y"],   # List of vertex labels to be loaded
        "v_extra_feats": ["train_mask","val_mask","test_mask"],     # Don't need any extra features for inference
        "output_format": "PyG",  # Using Pytorch Geometric format
        "batch_size": 64,        # Batch size for inference
        "num_neighbors": 10,     # Number of neighbors per vertex
        "num_hops": 2,           # How deep to go in the graph
        "shuffle": False         # Don't shuffle the data
    },
    "training_loader_config": {
        "v_in_feats": ["x"],
        "v_out_labels": ["y"],
        "v_extra_feats": ["train_mask","val_mask","test_mask"],
        "output_format": "PyG",
        "batch_size": 64, 
        "num_neighbors": 10, 
        "num_hops": 2,
        "shuffle": True
    },
    "optimizer_config": {
        "lr": 0.01,
        "weight_decay": 5e-4,
    },
    "connection_config": {
        "host": "http://3.22.188.182", 
        "graphname": "Cora", 
    }
}

### Write Parameters to JSON File
We will write the parameters dictionary to a JSON file so that we can easily access the parameters when creating the inference container.

In [ ]:
import json

json.dump(parameters, open("{}/config.json".format(source_directory), "w"))

## Train a GNN Model

### Load the Model
Here, we use some Python packaging tools to load the model. This is equivalent to writing `from source_directory.model import ModelName`.

Since `source_directory` and `ModelName` are unique to each developer's configs, we will use the `sys` package to import the model.

In [ ]:
import sys
sys.path.append(source_directory)

import model
GAT = getattr(model, parameters["model_name"])

In [ ]:
GAT

#### Instantiate the Model Class
Here, we use `kwargs` to pass in the parameters of the model from the parameters dictionary.

In [ ]:
gat = GAT(**parameters["model_config"])
gat

### Create Data Loaders
Here, we instantiate a connection to our TigerGraph database with `pyTigerGraph`. Then we create data loaders for training, validation, and testing datasets. We will use the **Neighbor Sampling** technique introduced in the GraphSAGE paper to generate batches of data.

In [ ]:
from pyTigerGraph import TigerGraphConnection

conn = TigerGraphConnection(**parameters["connection_config"])
conn.getToken(conn.createSecret())

('ns518b6ll2c23lelf04rlgtb84vlpf15', 1659648035, '2022-08-04 21:20:35')

In [ ]:
train_loader = conn.gds.neighborLoader(
    **parameters["training_loader_config"],
    filter_by="train_mask"
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.


In [ ]:
valid_loader = conn.gds.neighborLoader(
    **parameters["training_loader_config"],
    filter_by="val_mask"
)

In [ ]:
test_loader = conn.gds.neighborLoader(
    **parameters["training_loader_config"],
    filter_by="test_mask"
)

### Setup Optimizer
Here, we define the `Adam` optimizer and move the model to the correct device (CPU or GPU).

In [ ]:
import torch
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gat.to(device)

optimizer = torch.optim.Adam(
    gat.parameters(), **parameters["optimizer_config"]
)

### Train the Model

In [ ]:
from datetime import datetime
from pyTigerGraph.gds.metrics import Accumulator, Accuracy

In [ ]:
global_steps = 0
logs = {}
for epoch in range(10):
    # Train
    gat.train()
    epoch_train_loss = Accumulator()
    epoch_train_acc = Accuracy()
    for bid, batch in enumerate(train_loader):
        batchsize = batch.x.shape[0]
        batch.to(device)
        # Forward pass
        out = gat(batch)
        # Calculate loss
        loss = F.cross_entropy(out[batch.train_mask], batch.y[batch.train_mask])
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_train_loss.update(loss.item() * batchsize, batchsize)
        # Predict on training data
        with torch.no_grad():
            pred = out.argmax(dim=1)
            epoch_train_acc.update(pred[batch.train_mask], batch.y[batch.train_mask])
        # Log training status after each batch
        logs["loss"] = epoch_train_loss.mean
        logs["acc"] = epoch_train_acc.value
        print(
            "Epoch {}, Train Batch {}, Loss {:.4f}, Accuracy {:.4f}".format(
                epoch, bid, logs["loss"], logs["acc"]
            )
        )
        global_steps += 1
    # Evaluate
    gat.eval()
    epoch_val_loss = Accumulator()
    epoch_val_acc = Accuracy()
    for batch in valid_loader:
        batchsize = batch.x.shape[0]
        batch.to(device)
        with torch.no_grad():
            # Forward pass
            out = gat(batch)
            # Calculate loss
            valid_loss = F.cross_entropy(out[batch.val_mask], batch.y[batch.val_mask])
            epoch_val_loss.update(valid_loss.item() * batchsize, batchsize)
            # Prediction
            pred = out.argmax(dim=1)
            epoch_val_acc.update(pred[batch.val_mask], batch.y[batch.val_mask])
    # Log testing result after each epoch
    logs["val_loss"] = epoch_val_loss.mean
    logs["val_acc"] = epoch_val_acc.value
    print(
        "Epoch {}, Valid Loss {:.4f}, Valid Accuracy {:.4f}".format(
            epoch, logs["val_loss"], logs["val_acc"]
        )
    )

### Test the Model

In [ ]:
gat.eval()
acc = Accuracy()
for batch in test_loader:
    batch.to(device)
    with torch.no_grad():
        pred = gat(batch).argmax(dim=1)
        acc.update(pred[batch.test_mask], batch.y[batch.test_mask])
print("Accuracy: {:.4f}".format(acc.value))

### Save the Trained Model Weights

In [ ]:
torch.save(gat.state_dict(), "{}/model.pth".format(source_directory))

## Create Dockerfile

Google Vertex AI uses Docker containers in order to host models. We use a Dockerfile to build this container.

In [ ]:
%%writefile Dockerfile

FROM ubuntu:latest

# Install some basic utilities
RUN apt-get update && apt-get install -y \
    curl \
    ca-certificates \
    sudo \
    git \
    bzip2 \
    libx11-6 \
    wget \
    pip \
 && rm -rf /var/lib/apt/lists/*

WORKDIR /opt
# Set up the Conda environment
ENV CONDA_AUTO_UPDATE_CONDA=false \
    PATH=/opt/miniconda/bin:$PATH
COPY ./gat_cora/environment.yml /opt/environment.yml
RUN curl -sLo /opt/miniconda.sh https://repo.continuum.io/miniconda/Miniconda3-py39_4.11.0-Linux-x86_64.sh \
 && chmod +x /opt/miniconda.sh \
 && /opt/miniconda.sh -b -p /opt/miniconda \
 && rm /opt/miniconda.sh \
 && conda env update -n base -f /opt/environment.yml \
 && rm /opt/environment.yml \
 && conda clean -ya

 RUN pip install --no-index torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html \
 && pip install --no-index torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html \
 && pip install --no-index torch-cluster -f https://data.pyg.org/whl/torch-1.10.0+cu113.html \
 && pip install --no-index torch-spline-conv -f https://data.pyg.org/whl/torch-1.10.0+cu113.html \
 && pip install torch-geometric \
 && pip cache purge

# install - requirements.txt
COPY ./gat_cora/requirements.txt /tmp/requirements.txt
RUN python3 -m pip install -r /tmp/requirements.txt --quiet --no-cache-dir \
  && rm -f /tmp/requirements.txt

ENV TARGET_DIR /opt/kserve-demo
WORKDIR ${TARGET_DIR}
COPY ./gat_cora/ ${TARGET_DIR}/gat_cora/

ENTRYPOINT ["python3", "./gat_cora/main.py"]

## Define main.py File

This `main.py` file will load the model and start running an HTTP server for model inference within the Docker container.

In [ ]:
%%writefile $source_directory/main.py
import torch
import kserve
from google.cloud import storage
# from sklearn.externals import joblib
from kserve import Model, Storage
from kserve.model import ModelMissingError, InferenceError
from typing import Dict
import logging
import pyTigerGraph as tg
import os 
import sys
import json

logger = logging.getLogger(__name__)

class VertexClassifier(Model):
    def __init__(self, name: str, source_directory: str):
        super().__init__(name)
        self.name = name
        self.source_dir = source_directory
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
        # Load configuration JSON file
        with open(os.path.join(source_directory, "config.json")) as json_file:
            data = json.load(json_file)
            self.model_config = data["model_config"]
            connection_config = data["connection_config"]
            loader_config = data["infer_loader_config"]
            self.mdl_nm = data["model_name"]

        sys.path.append(source_directory)
        # Setup Connection to TigerGraph Database
        self.conn = tg.TigerGraphConnection(**connection_config)

        # Setup Inference Loader
        self.infer_loader = self.conn.gds.neighborLoader(**loader_config)

        # Setup Model
        self.model = self.load_model()

    def load(self):
        pass
    
    def load_model(self):
        import model
        mdl = getattr(model, self.mdl_nm)(**self.model_config)
        logger.info("Instantiated Model")
        with open(os.path.join(self.source_dir, "model.pth"), 'rb') as f:
            mdl.load_state_dict(torch.load(f))
        mdl.to(self.device).eval()
        logger.info("Loaded Model")
        return mdl

    def predict(self, request: Dict) -> Dict:
        input_nodes = request["instances"]
        input_ids = set([str(node['primary_id']) for node in input_nodes])
        logger.info(input_ids)
        data = self.infer_loader.fetch(input_nodes).to(self.device)
        logger.info (f"predicting {data}")
        with torch.no_grad():
            output = self.model(data)
        returnJSON = []
        for i in range(len(input_nodes)):
            returnJSON.append({input_nodes[i]["primary_id"]: list(output[i].tolist())})
        return json.dumps({"predictions": returnJSON})

if __name__ == "__main__":
    model_name = os.environ.get('K_SERVICE', "tg-gat-gcp-demo-predictor-default")
    model_name = '-'.join(model_name.split('-')[:-2]) # removing suffix "-predictor-default"
    print(model_name)
    logging.info(f"Starting model '{model_name}'")
    model = VertexClassifier(model_name, "./gat_cora/")
    kserve.ModelServer(http_port=8080).start([model])


## Write requirements.txt File

In [ ]:
%%writefile $source_directory/requirements.txt

# kubeflow packages
kfp==1.6.3
kfp-server-api==1.6.0
kserve==0.8

# common packages
#bokeh==2.3.2
#cloudpickle==1.6.0
#dill==0.3.4
#pandas==1.2.4

# pytorch packages
#fastai==2.4
class-resolver==0.3.9

# TigerGraph
pyTigerGraph[gds]

## Write environment.yml File

In [ ]:
%%writefile $source_directory/environment.yml
name: base
dependencies:
- numpy=1.21.2
- pip=21.2.4
- python=3.9.7
- pytorch::pytorch=1.10.0=py3.9_cuda11.3_cudnn8.2.0_0
- scipy=1.7.1
- cloudpickle=2.0.0  

## Build Docker Image
Using the Dockerfile defined above, we will build the Docker image for inference.

In [ ]:
!docker build --rm --platform linux/amd64 -t kserve-base:1.0 . 

## Run Docker Image Locally

In [ ]:
!docker run -p 8080:8080 kserve-base:1.0

## Test Local Deployment

In [ ]:
import requests

data = {"instances": [{"primary_id": 7, "type": "Paper"}, {"primary_id": 17, "type": "Paper"}, {"primary_id": 27, "type": "Paper"}, {"primary_id": 37, "type": "Paper"}]}

resp = requests.post("http://localhost:8080/v1/models/tg-gat-gcp-demo:predict", json=data)

In [ ]:
resp.json()

In [ ]:
requests.get("http://localhost:8080/v1/models").text

## Upload to GCP

In [ ]:
!docker tag kserve-base:1.0 us-central1-docker.pkg.dev/tigergraph-ml/gnn-inference/cora-gat-inference:latest

In [ ]:
!sudo docker push us-central1-docker.pkg.dev/tigergraph-ml/gnn-inference/cora-gat-inference:latest

## Deploy Model

In [ ]:
!gcloud ai models upload \
  --region=us-central1 \
  --display-name=cora-gat \
  --container-image-uri=us-central1-docker.pkg.dev/tigergraph-ml/gnn-inference/cora-gat-inference:latest \
  --container-health-route=/v1/models \
  --container-predict-route=/v1/models/tg-gat-gcp-demo:predict


In [ ]:
!gcloud ai models list --region=us-central1 

## Create Endpoint

In [ ]:
!gcloud ai endpoints create --region=us-central1 --display-name=coragat

In [ ]:
!gcloud ai endpoints list --region=us-central1 --filter=display_name=coragat

## Deploy Model to Endpoint

In [ ]:
!gcloud ai endpoints deploy-model YOUR_ENDPOINT_ID\
  --region=us-central1 \
  --model=YOUR_MODEL_ID \
  --display-name=coragat \
  --machine-type=n1-standard-2 \
  --min-replica-count=1 \
  --max-replica-count=5 \
  --traffic-split=0=100


## Run Prediction

In [ ]:
import json
json.dump(data, open("request.json", "w"))

In [ ]:
!ENDPOINT_ID="YOUR_ENDPOINT_ID"
!PROJECT_ID="tigergraph-ml"
!INPUT_DATA_FILE="request.json" 

In [ ]:
!curl \                           
-X POST \                 
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-central1/endpoints/${ENDPOINT_ID}:predict \
-d "@${INPUT_DATA_FILE}"